In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import csv
import numpy as np

In [2]:
import numpy as np


sentiment_data = 'sentiment-data'


def getWordVectorDict():
    reader = csv.reader(open(sentiment_data +'/word-vectors-refine.txt'))

    word_vector_dict = {}
    for row in reader:
        key = row[0]
        if key in word_vector_dict:
            # implement your duplicate row handling here
            pass
        word_vector_dict[key] = np.array(row[1:])
    return word_vector_dict

def getPaddedSentenceMatrix(sentenceMatrix):
    wordCount = 100
    return np.vstack((sentenceMatrix, np.zeros((wordCount - np.shape(sentenceMatrix)[0],np.shape(sentenceMatrix)[1]), dtype=np.float32)))

def getVectorForSentence(sentence, word_vec_dict):
    sentence_matrix = []
    for word in sentence.split(' '):
        word_vec = word_vec_dict[word]
        if(len(sentence_matrix) == 0):
            sentence_matrix = word_vec
        else:
            sentence_matrix = np.vstack((sentence_matrix,word_vec))
    return getPaddedSentenceMatrix(sentence_matrix)

In [3]:
vocabulary_size = 28
n_inputs = 50
n_steps = 100
n_neurons = 150
n_outputs = 2
learning_rate = 0.01

init = tf.global_variables_initializer()


word_vector_size = 50;
time_steps = 100;
num_classes = 2
batch_size = 1000;
n_iterations = 10;
hidden_layer_size = 50

In [4]:

def getData(fileName):
    reader = csv.reader(open(sentiment_data +'/' + fileName))
    trainingData = []
    for row in reader:
        data = {}
        data['label'] =  1 if row[0] == 'postive' else 0
        data['sentence'] = row[1:]
        trainingData.append(data)
    return trainingData

In [5]:
word_vec_dict = getWordVectorDict()


def transform(row):
    return row['label'], getVectorForSentence(row['sentence'][0], word_vec_dict)

In [6]:
training_data = getData('train.csv')
training_rows  = map(lambda row: transform(row), training_data)
training_data = map(lambda row: row[1], training_rows)
training_labels = map(lambda row: row[0], training_rows)
#test_data = map(lambda row: transform(row), getData('test.csv'))
test_data = getData('test.csv')
test_rows  = map(lambda row: transform(row), test_data)
test_data = map(lambda row: row[1], test_rows)
test_labels = map(lambda row: row[0], test_rows)
print training_data[0]

[['0.68491' '0.32385' '-0.11592' ..., '0.17874' '-0.1693' '0.062375']
 ['0.96193' '0.012516' '0.21733' ..., '0.14032' '-0.38468' '-0.38712']
 ['0.6008' '0.18044' '0.078339' ..., '-0.016404' '-0.65372' '-0.38255']
 ..., 
 ['0.88387' '-0.14199' '0.13566' ..., '0.52711' '-0.20148' '0.0095952']
 ['-0.0010919' '0.33324' '0.35743' ..., '-0.45697' '-0.048969' '1.1316']
 ['-0.55114' '-0.16296' '-0.95494' ..., '-1.0346' '-0.25143' '1.4836']]


In [ ]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation = tf.tanh)
outputs , states = tf.nn.dynamic_rnn(basic_cell, X, dtype= tf.float32)
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels= y , logits= logits)
loss = tf.reduce_mean(xentropy)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer = tf.train.RMSPropOptimizer(0.1, 0.9, 0.01)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:

init = tf.global_variables_initializer()


batch_size = 100;
n_iterations = 20;

with tf.Session() as sess:
    init.run()
    for epoch in range(n_iterations):
        print "Running epoch:",epoch
        for j in range(len(training_data)/batch_size):
            startIndex = j*batch_size
            endIndex = startIndex + batch_size
            batch_x = np.array(training_data[startIndex : endIndex]).reshape((-1,time_steps, word_vector_size))
            batch_y = np.array(training_labels[startIndex : endIndex]).reshape(batch_size)
            sess.run(training_op, feed_dict={X: batch_x, y: batch_y})
        acc_train = accuracy.eval(feed_dict={X: np.array(training_data), y: np.array(training_labels)})
        acc_test = accuracy.eval(feed_dict={X: np.array(test_data), y: np.array(test_labels)})
        print(epoch,"Train accuracy:", acc_train, " Test accuracy:", acc_test)

    print ("Testing Accuracy:",
    sess.run(accuracy, feed_dict={X: test_data, y: test_labels}))

Running epoch: 0
(0, 'Train accuracy:', 0.49875, ' Test accuracy:', 0.50199997)
Running epoch: 1
(1, 'Train accuracy:', 0.49994999, ' Test accuracy:', 0.50239998)
Running epoch: 2
(2, 'Train accuracy:', 0.49959999, ' Test accuracy:', 0.50279999)
Running epoch: 3
